In [ ]:
import sys
import os
from natsort import natsorted
import stereo as st
from stereo.core.ms_data import MSData
from stereo.core.ms_pipeline import slice_generator
import pandas as pd
import tarfile
import pickle
import csv

import numpy as np

import warnings
warnings.filterwarnings('ignore')

Bin = 20
n = 5
sample = 'E1C1D6B1'
outdir = '/data/work/output/Clustering/Stereopy/ITAG4.1/with_MT_CP/Multi_sample/Combind/' + sample + '/Clustering/scaled_Normalization_Other_Unpaste_Union/'
os.makedirs(outdir, exist_ok=True)

outdir1 = outdir + 'Makergenes/'
os.makedirs(outdir1, exist_ok=True)

folder_target_log2fc = outdir1 + 'log2fc_all/'
os.makedirs(folder_target_log2fc, exist_ok=True) 

folder_target_score = outdir1 + 'scores_all/'
os.makedirs(folder_target_score, exist_ok=True) 

outdir2 = outdir + 'hotspot/'
os.makedirs(outdir2, exist_ok=True)


# reading all slices
ms_data = MSData(_var_type='union') 

data = st.io.read_gef('/data/input/W202505250000259/SAW-ST-V8-reanalyze-lasso/lasso/L1U1/SS200000144TL_E1.L1U1.label.gef', bin_size=Bin)
ms_data['E1_L1U1'] = data
data = st.io.read_gef('/data/input/W202505250000256/SAW-ST-V8-reanalyze-lasso/lasso/L1U2/SS200000144TL_E1.L1U2.label.gef', bin_size=Bin)
ms_data['E1_L1U2'] = data
data = st.io.read_gef('/data/input/W202505250000258/SAW-ST-V8-reanalyze-lasso/lasso/L1U3/SS200000144TL_E1.L1U3.label.gef', bin_size=Bin)
ms_data['E1_L1U3'] = data
data = st.io.read_gef('/data/input/W202505250000255/SAW-ST-V8-reanalyze-lasso/lasso/L2U1/SS200000144TL_E1.L2U1.label.gef', bin_size=Bin)
ms_data['E1_L2U1'] = data
data = st.io.read_gef('/data/input/W202505250000257/SAW-ST-V8-reanalyze-lasso/lasso/L2U2/SS200000144TL_E1.L2U2.label.gef', bin_size=Bin)
ms_data['E1_L2U2'] = data

data = st.io.read_gef('/data/input/W202505250002038/SAW-ST-V8-reanalyze-lasso/lasso/LT_SAM/SS200000155TR_C1.LT_SAM.label.gef', bin_type='bins',bin_size=Bin)
ms_data['C1_LT'] = data
data = st.io.read_gef('/data/input/W202505250002042/SAW-ST-V8-reanalyze-lasso/lasso/LB_SAM/SS200000155TR_C1.LB_SAM.label.gef', bin_type='bins',bin_size=Bin)
ms_data['C1_LB'] = data
data = st.io.read_gef('/data/input/W202505250002039/SAW-ST-V8-reanalyze-lasso/lasso/RB_SAM/SS200000155TR_C1.RB_SAM.label.gef', bin_type='bins',bin_size=Bin)
ms_data['C1_RB'] = data

data = st.io.read_gef('/data/input/W202505230000352/SAW-ST-V8-reanalyze-lasso/lasso/L1U1/SS200000144TL_D6.L1U1.label.gef', bin_size=Bin)
ms_data['D6_L1U1'] = data
data = st.io.read_gef('/data/input/W202505230000351/SAW-ST-V8-reanalyze-lasso/lasso/L2U1/SS200000144TL_D6.L2U1.label.gef', bin_size=Bin)
ms_data['D6_L2U1'] = data
data = st.io.read_gef('/data/input/W202505230000350/SAW-ST-V8-reanalyze-lasso/lasso/L2U2/SS200000144TL_D6.L2U2.label.gef', bin_size=Bin)
ms_data['D6_L2U2'] = data
data = st.io.read_gef('/data/input/W202505220011066/SAW-ST-V8-reanalyze-lasso/lasso/L2U3/SS200000144TL_D6.L2U3.label.gef', bin_size=Bin)
ms_data['D6_L2U3'] = data

data = st.io.read_gef('/data/input/W202505280001492/SAW-ST-V8-reanalyze-lasso/lasso/LT/SS200000163TR_B1.LT.label.gef', bin_size=Bin) 
ms_data['B1_LT'] = data
data = st.io.read_gef('/data/input/W202505280001495/SAW-ST-V8-reanalyze-lasso/lasso/RB/SS200000163TR_B1.RB.label.gef', bin_size=Bin) 
ms_data['B1_RB'] = data
data = st.io.read_gef('/data/input/W202505280001494/SAW-ST-V8-reanalyze-lasso/lasso/RT/SS200000163TR_B1.RT.label.gef', bin_size=Bin)
ms_data['B1_RT'] = data
data = st.io.read_gef('/data/input/W202505280001493/SAW-ST-V8-reanalyze-lasso/lasso/LB/SS200000163TR_B1.LB.label.gef', bin_size=Bin) 
ms_data['B1_LB'] = data

After loading sorted data into MSData object, just type it to obtain basic information.


ms_data.tl.set_scope_and_mode(
    scope=slice_generator[:],
    mode='isolated'
)

ms_data.tl.raw_checkpoint()
ms_data.tl.raw


ms_data.tl.set_scope_and_mode(
    scope=slice_generator[:],
    mode='integrate'
)

ms_data.tl.raw_checkpoint()
ms_data.tl.raw

ms_data.integrate() 
ms_data

st.io.stereo_to_anndata(ms_data.merged_data, flavor='scanpy', sample_id=sample, reindex=False, output=outdir + sample + '_union_anndata.h5ad', base_adata=None, split_batches=False, compression='gzip')

ms_data.tl.set_scope_and_mode(
    scope=slice_generator[:],
    mode='integrate'
)

ms_data.tl.cal_qc(scope=slice_generator[:],mode='integrate')
ms_data


ms_data.tl.set_scope_and_mode(
    scope=slice_generator[:],
    mode='integrate'
)
ms_data.tl.normalize_total(target_sum=10000)
ms_data.tl.log1p()

ms_data.tl.highly_variable_genes(
        min_mean=0.0125,
        max_mean=3,
        min_disp=0.5,
        n_top_genes=2000, 
        res_key='highly_variable_genes',
        scope=slice_generator[:],
        mode='integrate'
        ) 

Scale each gene to unit variance. Clip values exceeding standard deviation 10. If `data.tl.scale(zero_center=False)` is used, sparse matrix will be used for calculation, which can greatly reduce the memory required for running.


ms_data.tl.scale(max_value=10, zero_center=True) 

ms_data.tl.pca(
        use_highly_genes=False, 
        n_pcs=30, 
        res_key='pca',
        scope=slice_generator[:],
        mode='integrate'
        )

ms_data.tl.neighbors(
        pca_res_key='pca', 
        n_pcs=30, 
        res_key='neighbors',
        scope=slice_generator[:],
        mode='integrate'
        )

ms_data.tl.umap(
            pca_res_key='pca', 
            neighbors_res_key='neighbors', 
            res_key='umap',
            scope=slice_generator[:],
            mode='integrate'
            )

ms_data.tl.leiden(
            neighbors_res_key='neighbors', 
            res_key='leiden',
            scope=slice_generator[:],
            mode='integrate'
            )

ms_data.plt.umap(
            cluster_key='leiden',
            res_key='umap',
            scope=slice_generator[:],
            mode='integrate',
            out_path = outdir +  sample + '_cluster_umap_smalldotsize.pdf',
            out_dpi = 300
            )

ms_data.plt.cluster_scatter(
    res_key='leiden',
    scope=slice_generator[:],
    mode='integrate',
    plotting_scale_width=1000,       
    dot_size = 1,
    reorganize_coordinate=n,         
    horizontal_offset_additional=500, 
    out_path = outdir +  sample + '_cluster_scatter_smalldotsize.pdf',
    out_dpi = 300    
    )   